In [2]:
#Importing all necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
import seaborn as sns
df = pd.read_csv('data/train.csv')

In [3]:
df

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0,0.95577,0.0,8.14,0,0.538,6.047,88.8,4.4534,4,307.0,21.0,306.38,17.28,14.8
1,1,0.02875,28.0,15.04,0,0.464,6.211,28.9,3.6659,4,270.0,18.2,396.33,6.21,25.0
2,2,1.22358,0.0,19.58,0,0.605,6.943,97.4,1.8773,5,403.0,14.7,363.43,4.59,41.3
3,3,5.66637,0.0,18.10,0,0.740,6.219,100.0,2.0048,24,666.0,20.2,395.69,16.59,18.4
4,4,0.04544,0.0,3.24,0,0.460,6.144,32.2,5.8736,4,430.0,16.9,368.57,9.09,19.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,0.03615,80.0,4.95,0,0.411,6.630,23.4,5.1167,4,245.0,19.2,396.90,4.70,27.9
396,396,0.17505,0.0,5.96,0,0.499,5.966,30.2,3.8473,5,279.0,19.2,393.43,10.13,24.7
397,397,6.65492,0.0,18.10,0,0.713,6.317,83.0,2.7344,24,666.0,20.2,396.90,13.99,19.5
398,398,0.13117,0.0,8.56,0,0.520,6.127,85.2,2.1224,5,384.0,20.9,387.69,14.09,20.4


In [4]:
#analysing the file 
print(df.info())
print(df.describe())
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       400 non-null    int64  
 1   CRIM     400 non-null    float64
 2   ZN       400 non-null    float64
 3   INDUS    400 non-null    float64
 4   CHAS     400 non-null    int64  
 5   NOX      400 non-null    float64
 6   RM       400 non-null    float64
 7   AGE      400 non-null    float64
 8   DIS      400 non-null    float64
 9   RAD      400 non-null    int64  
 10  TAX      400 non-null    float64
 11  PTRATIO  400 non-null    float64
 12  B        400 non-null    float64
 13  LSTAT    400 non-null    float64
 14  MEDV     400 non-null    float64
dtypes: float64(12), int64(3)
memory usage: 47.0 KB
None
               ID        CRIM          ZN       INDUS        CHAS         NOX  \
count  400.000000  400.000000  400.000000  400.000000  400.000000  400.000000   
mean   199.500000    3.757191 

In [17]:
df_test =pd.read_csv("data/test.csv")

In [12]:
#loading Datas
PHI = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1, 14)))
PHI_test = np.loadtxt('data/test.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1,14)))
Y = np.loadtxt('data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=14, ndmin=2)
PHI_test = np.concatenate((PHI_test, np.ones((105, 1))), axis=1)
PHI = np.concatenate((PHI, np.ones((400, 1))), axis=1)


In [13]:
#Scaling the data
from sklearn.preprocessing import MinMaxScaler
func = MinMaxScaler(feature_range=(0,1))
func.fit(PHI)
PHI = func.transform(PHI)
PHI_test = func.transform(PHI_test)
Y = np.log(Y)

In [18]:
#building the model
P_values = {'output_l2.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }

def error_change(p, PHI, w):
    if p == 2:
        deltaw = (2 * (np.dot(np.dot(np.transpose(PHI), PHI), w) - np.dot(np.transpose(PHI), Y)) + lambda1 * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        deltaw = (2 * (np.dot(np.dot(np.transpose(PHI), PHI), w) - np.dot(np.transpose(PHI), Y)) + lambda1 * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return deltaw


for (fname, p) in P_values.items():
    # Set w to 0's
    w = np.zeros((14, 1))
    lambda1 = 0.2 #hyperparameter 
    t = 0.00012 #step size

    w_new = w - t * error_change(p, PHI , w)

    i = 0
    
    while(np.linalg.norm(w_new-w) > 10 ** -10):
        w = w_new
        w_new = w - t * error_change(p, PHI, w)
        i = i + 1
  
    id_test = np.loadtxt('data/test.csv', dtype='int', delimiter=',', skiprows=1, usecols=0, ndmin=2)

    # predecting y for test data
    y_test = np.exp(np.dot(PHI_test, w_new))
    df_test[fname] = y_test
    # Save the ids and y
    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1),
               delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')


In [19]:
df_test.head()

,ID,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,output_l2.csv,output_p1.csv,output_p2.csv,output_p3.csv
0,0,0.10612,30.0,4.93,0,0.428,6.095,65.1,6.3361,6,300.0,16.6,394.62,12.40,21.308879,21.227376,21.212540,21.202891
1,1,0.34109,0.0,7.38,0,0.493,6.415,40.1,4.7211,5,287.0,19.6,396.90,6.12,22.882087,22.914278,22.932679,22.948583
2,2,12.24720,0.0,18.10,0,0.584,5.837,59.7,1.9976,24,666.0,20.2,24.65,15.69,5.273359,5.275525,5.286807,5.292653
3,3,0.22489,12.5,7.87,0,0.524,6.377,94.3,6.3467,5,311.0,15.2,392.52,20.45,29.036429,29.025491,29.078914,29.116071
4,4,1.80028,0.0,19.58,0,0.605,5.877,79.2,2.4259,5,403.0,14.7,227.61,12.14,10.951298,10.976464,10.997946,11.016213


### Conclusion
1) Comparison of performance of p1=1.75, p2=1.5, p3=1.3 have been performed and saved to the output file. 

2) Error from L2 Gradient descent were in close range compared to other values of p.
